In [1]:
import librosa
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
from IPython.display import Audio
import numpy as np
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
from sklearn.metrics import confusion_matrix
import IPython.display as ipd  # To play sound in the notebook
import os # interface with underlying OS that python is running on
import sys

In [2]:
# LOAD IN FILE
main_dir = 'D:/Makaleler/makale19 speech recog/ravdes/ravdes/Audio_Speech_Actors_01-24/'
#main_dir ='D:/Makaleler/makale19 speech recog/ravdes/ex/'
sub_dir = os.listdir(main_dir)
x, sr = librosa.load(main_dir+sub_dir[0]+'/03-01-06-01-02-02-01.wav')
# PLAY any AUDIO FILE
#librosa.output.write_wav('MaleNeutral.wav', x, sr)
Audio(data=x, rate=sr)

In [3]:
def extract_feature(file_name, offst=0.5):
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast',offset=offst)
    
    stft = np.abs(librosa.stft(X))
    chroma_cq = librosa.feature.chroma_cqt(y=X, sr=sample_rate)
    cqt=np.mean(chroma_cq,axis=1)
    
    #chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate,n_fft=2048).T,axis=0)
    #chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate,n_mels=128,fmax=8000).T,axis=0)
    #mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,cqt,mel,contrast,tonnetz

def extract_featurev2(file_name):
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast', sr=None)   
    #mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=64).T, axis=0)
    C = np.abs(librosa.cqt(X, sr=sr, fmin=librosa.note_to_hz('C2'), n_bins=60))
    #d=librosa.amplitude_to_db(C)
    d=librosa.power_to_db(C)
    cqt = np.mean(d,axis=1)
    return cqt

In [4]:
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [5]:
# CREATE FUNCTION TO EXTRACT EMOTION NUMBER, ACTOR AND GENDER LABEL
emotion = []
gender = []
actor = []
file_path = []
file_pathShort = []
a= []
b= []
c= []
d= []
e= []

X4= pd.DataFrame()
cc=0
for i in sub_dir:
    filename = os.listdir(main_dir + i) #iterate over Actor folders
    #print(filename)
    for f in filename: 
        part = f.split('.')[0].split('-')
        emotion.append(int(part[2]))
        actor.append(int(part[6]))
        bg = int(part[6])
        if bg%2 == 0:
            bg = 0 #"female"
        else:
            bg = 1 #"male"
        gender.append(bg)
        file_path.append(main_dir + i + '/' + f)
        file_pathShort.append(i + '/' + f)
        a,b,c,d,e=extract_feature(main_dir + i + '/' + f)
        #a=extract_feature(main_dir + i + '/' + f)
        tot= []
        for x in a:tot.append(x)
        for x in b:tot.append(x)
        for x in c:tot.append(x)
        for x in d:tot.append(x)
        for x in e:tot.append(x)
        
        #X4[cc]=pd.DataFrame([tot])
        X4[cc]=np.asarray(tot)
        cc+=1;
        #print(tot)
    
# PUT EXTRACTED LABELS WITH FILEPATH INTO DATAFRAME
audio_df = pd.DataFrame(emotion)
#audio_df = audio_df.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'})
audio_df = pd.concat([pd.DataFrame(gender),audio_df,pd.DataFrame(actor)],axis=1)
audio_df.columns = ['gender','emotion','actor']
audio_df = pd.concat([audio_df,pd.DataFrame(file_pathShort, columns = ['pathShort']),
                      pd.DataFrame(file_path, columns = ['path'])],axis=1)
len(tot)

C:\Users\ender\anaconda3\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=966
  return f(*args, **kwargs)
C:\Users\ender\anaconda3\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=966
  return f(*args, **kwargs)
C:\Users\ender\anaconda3\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=978
  return f(*args, **kwargs)
C:\Users\ender\anaconda3\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=955
  return f(*args, **kwargs)
C:\Users\ender\anaconda3\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=920
  return f(*args, **kwargs)
C:\Users\ender\anaconda3\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=989
  return

193

In [21]:
len(a),len(b),len(c),len(d),len(e)

(40, 12, 128, 7, 6)

In [22]:
X6= pd.DataFrame(X4.T)

In [23]:
type(X6), X6.shape

(pandas.core.frame.DataFrame, (1440, 193))

In [31]:
X7=X6.copy()

In [32]:
X7

,0,1,2,3,4,5,6,7,8,9,...,183,184,185,186,187,188,189,190,191,192
0,-672.884766,68.927429,-2.939275,18.483477,5.500031,4.235157,-7.293960,-0.283840,-16.094763,-0.344846,...,14.395096,15.385672,17.478247,39.363655,-0.049597,0.025619,-0.044120,-0.080482,0.023776,0.009147
1,-662.876282,67.128922,-6.399641,21.694197,5.777572,2.545914,-5.730112,-3.583304,-17.242125,0.736955,...,15.058568,15.371827,17.521408,38.392487,-0.079907,0.031592,-0.065873,-0.072478,0.026761,-0.005743
2,-664.416016,70.183823,-4.428379,18.341318,1.958050,3.746187,-6.504058,-4.076615,-15.753084,-2.368206,...,14.858060,15.617201,17.309580,38.543594,-0.028561,-0.009325,0.003015,0.027282,0.000309,0.030571
3,-660.299988,68.741692,-0.525135,19.285566,4.039678,6.074459,-5.181080,-4.279423,-15.026144,-1.570145,...,15.143183,15.533297,16.433031,38.045551,-0.060979,0.011434,0.002199,-0.025374,0.000205,0.017161
4,-699.018127,76.061127,-0.964818,20.963692,5.114833,5.603961,-7.761878,-1.267789,-14.995935,-2.746219,...,14.530594,15.960327,16.956637,38.413870,-0.072313,0.009371,0.032281,-0.116258,0.025949,0.002369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,-558.012939,33.520958,-25.292545,2.852235,-22.460224,-6.676312,-22.499645,-13.656723,-11.484236,-4.583565,...,16.289653,18.007045,17.446850,39.516731,-0.019159,0.014506,0.020639,0.051117,-0.005122,0.022333
1436,-509.447357,48.729256,-22.362350,-1.398088,-22.556530,-10.647760,-17.144878,-8.896639,-14.071460,0.397057,...,16.132876,16.953430,17.792025,38.693254,-0.041292,0.017657,0.014354,0.016839,0.016844,-0.000725
1437,-517.197327,39.712631,-26.507496,2.247680,-22.035622,-14.673651,-19.214752,-9.156968,-12.672955,3.738830,...,16.041746,16.897427,18.422814,38.928553,-0.031408,0.033740,-0.049822,0.008912,0.007382,0.013001
1438,-463.539062,34.857422,-13.660452,8.858074,-19.420992,0.439111,-17.810989,-4.469375,-7.315601,0.363834,...,16.245986,17.837694,17.299424,39.203758,-0.046567,0.026685,-0.043579,-0.061446,0.006565,-0.008730


In [33]:
X7.shape

(1440, 193)

In [34]:
X8=pd.concat([X7,pd.DataFrame(gender),pd.DataFrame(emotion)],axis=1)

In [35]:
X8.head()

,0,1,2,3,4,5,6,7,8,9,...,185,186,187,188,189,190,191,192,0,0
0,-672.884766,68.927429,-2.939275,18.483477,5.500031,4.235157,-7.293960,-0.283840,-16.094763,-0.344846,...,17.478247,39.363655,-0.049597,0.025619,-0.044120,-0.080482,0.023776,0.009147,1,1
1,-662.876282,67.128922,-6.399641,21.694197,5.777572,2.545914,-5.730112,-3.583304,-17.242125,0.736955,...,17.521408,38.392487,-0.079907,0.031592,-0.065873,-0.072478,0.026761,-0.005743,1,1
2,-664.416016,70.183823,-4.428379,18.341318,1.958050,3.746187,-6.504058,-4.076615,-15.753084,-2.368206,...,17.309580,38.543594,-0.028561,-0.009325,0.003015,0.027282,0.000309,0.030571,1,1
3,-660.299988,68.741692,-0.525135,19.285566,4.039678,6.074459,-5.181080,-4.279423,-15.026144,-1.570145,...,16.433031,38.045551,-0.060979,0.011434,0.002199,-0.025374,0.000205,0.017161,1,1
4,-699.018127,76.061127,-0.964818,20.963692,5.114833,5.603961,-7.761878,-1.267789,-14.995935,-2.746219,...,16.956637,38.413870,-0.072313,0.009371,0.032281,-0.116258,0.025949,0.002369,1,2


In [36]:
X8.tail()

,0,1,2,3,4,5,6,7,8,9,...,185,186,187,188,189,190,191,192,0,0
1435,-558.012939,33.520958,-25.292545,2.852235,-22.460224,-6.676312,-22.499645,-13.656723,-11.484236,-4.583565,...,17.446850,39.516731,-0.019159,0.014506,0.020639,0.051117,-0.005122,0.022333,0,8
1436,-509.447357,48.729256,-22.362350,-1.398088,-22.556530,-10.647760,-17.144878,-8.896639,-14.071460,0.397057,...,17.792025,38.693254,-0.041292,0.017657,0.014354,0.016839,0.016844,-0.000725,0,8
1437,-517.197327,39.712631,-26.507496,2.247680,-22.035622,-14.673651,-19.214752,-9.156968,-12.672955,3.738830,...,18.422814,38.928553,-0.031408,0.033740,-0.049822,0.008912,0.007382,0.013001,0,8
1438,-463.539062,34.857422,-13.660452,8.858074,-19.420992,0.439111,-17.810989,-4.469375,-7.315601,0.363834,...,17.299424,39.203758,-0.046567,0.026685,-0.043579,-0.061446,0.006565,-0.008730,0,8
1439,-498.993683,38.967422,-7.731936,3.212382,-10.446373,-1.859776,-16.630434,-1.310444,-14.403450,2.125301,...,16.524718,39.351362,-0.066797,0.060365,-0.020957,-0.046365,-0.011488,0.018706,0,8


In [37]:
X7.to_csv("featureNormal_Ek.csv",index=False,header=True)
#audio_df.to_csv("audioELM.csv",index=False,header=True)